In [362]:
import pandas as pd
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
stops = stopwords.words()

In [342]:
def _file(file):
    file = open(file, "r", encoding="utf-8")
    text = file.read()
    file.close()
    return text

def clean_text(text, vocab, stops):
	res = ''.join(re.findall(r'[a-zA-z]| ', text))
	tokens = [i.lower() for i in res.split() if i in vocab and i not in stops]
	return ' '.join(tokens)

In [343]:
vocab= _file('vocab3.txt')
df = pd.read_csv('train.csv')

df['text'] = df['text'].apply(clean_text, args=(vocab, stops))


In [375]:
index = df.index
df['random_number'] = np.random.randn(len(index))
train = df[df['random_number'] <= 1]
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train['text'])
X_train = train_matrix
y_train = train['answer']
lr = LogisticRegression(solver='lbfgs', max_iter=1000) 

lr.fit(X_train, y_train)
lr.score(X_test, y_test)



0.8429578806937298

In [379]:
df_test = pd.read_csv('test.csv')
df_test['text'] = df_test['text'].apply(clean_text, args=(vocab, stops))
y_test = vectorizer.fit_transform(df_test['text'])
lr.predict(y_test)

ValueError: X has 7734 features, but LogisticRegression is expecting 7594 features as input.